# Tarea 6. Embeddings and CNNs

Guillermo Segura Gomez

## Word2Vec

**Estudie los papers de word2vec (a y b). Consteste las siguientes preguntas en sus propias palabras.**

**1. ¿Describa en sus propias palabras la estrategia de selección de palabras dentro de la ventana de contexto en w2v? Explique porque se hace así y cual es la intuición.**

Lo primero es definir la ventana de contexto. En el articulo *Efficient Estimation of Word Representations in Vector Space* [2], Mikolov propone un modelo llamado **continuous skip-gram model** en el que trata de maximizar la clasificación de una palabra basado en otra palabra de la misma oración. Es decir, y cito textualmente "Se usa cada palabra actual como entrada para un clasificador log-lineal con una capa de proyección continua y predice palabras dentro de un cierto rango antes y después de la palabra actual". El rango o distancia maxima entre las palabras que se pueden seleccionar para este análisis se conoce como **ventana de contexto**, representada por la letra $C$. La estrategia que se utiliza para la selección de palabras dentro de la ventana es una estrategia aleatoria. Para cada palabra del conjunto de entrenamiento se selecciona de manera random un numero entero $R$ en el rango $[1,C]$, luego se seleccionan $R$ palabras anteriores y $R$ palabras posteriores a la palabra objetivo como el contexto actual para ese ejemplo de entrenamiento. La intuición detrás de esto es que no todas las palabras dentro de la ventana de contexto son seleccionadas con igual probabilidad, es necesario agregar este comportamiento aleatorio para evitar que el modelo se sobreajuste a ciertas ventanas de contexto y poder capturar la mayor cantidad de información sintáctica y semántica posible. De otra forma tendríamos un modelo de n-gramas simple, el cual hemos visto que presenta complicaciones para modelar el lenguaje a profundidad. 

**2. ¿Qué estrategia se usa para construir frases de palabras y construir un solo vector para conceptos basados en más de un token?**

En el articulo *Distributed Representations of Words and Phrases and their Compositionality* [1], Mikolov habla acerca de que el modelo de skip-grams puede aprender conceptos mas profundos que una sola palabra. Y es que por lo que entiendo la idea de los modelos tipo embeddings es *linealidad* el lenguaje, es decir, poder encontrar representaciones de palabras con las cuales podamos mediante operaciones vectoriales lineales, podamos llegar a otros vectores de palabras que tengan sentido semántico; por ejemplo *vector("King")* - *vector(Man)* + *vector(Woman)* = *vector(Queen)*. Es asi, que hay muchas *frases* cuyo significado no es una composición simple de significados de palabras individuales. Para que el modelo pueda aprender este tipo de frases, se tiene que realizar un preproceso para poder construir estas frases en el corpus y que cada frase tenga un token único. Me parece que la estrategia que se sigue es primero buscar los conjuntos de palabras que por si mismo aparecen de manera muy rara en contextos únicos. Por ejemplo "New York Times" que involucra tres palabras. Luego es necesario construir un token único de esta palabra y agregarlo al corpus. Para construir el token único se pueden agregar guiones para tener "New_York_Times" y con esto poder conseguir frases de palabras que se aprendan por si mismas en el modelo.

**3. ¿Según el autor de w2v, cuales podrían ser las diferencias o ventajas/desventajas de CBOW y Skipgram?**

En el articulo donde se proponen los modelos de embeddings [2], nos damos cuenta de que son modelos con ideas similares, aunque opuestas. El modelo de CBOW predice la palabra actual basándose en el contexto de su alrededor mientras que el modelo se skip-grams se intenta predecir las palabras de contexto a partir de la palabra actual. En general el modelo CBOW es el más rápido entrenar. Sin embargo el modelo skip-grams es más efectivo para aprender representaciones de alta calidad para palabras raras, ya que trata cada instancia de la palabra objetivo como un ejemplo de entrenamiento único. Si se desea escoger uno de los modelos se tiene que hacer en base a las necesidades y disponibilidad de hardware que se tenga, CBOW es mas eficiente, y funciona bien para conjuntos peque;os de datos, mientas que skip-gram es brutal para conjuntos grandes y para aprender palabras extra;as en el contexto.

**4. ¿Cuales son las diferencias entre usar Hierachical Softmax, Negative Sampling y NCE? ¿Cuál recomienda el autor y por qué?**

Para responder esta pregunta, es necesario conocer como se calculan las probabilidades de una red neuronal. Para una neurona simple, una red neuronal recibe una cantidad de inputs y los procesa mediante la multiplicación por los pesos y la suma del bias. Luego se aplica una función no lineal, y finalmente se calcula la probabilidad mapeando la salida de la función no lineal que va de $(-\infty, \infty)$ a un valor entre $[0,1]$, una probabilidad. Este mapeo se hace utilizando la función softmax. Sin embargo hay complicaciones a la hora de manejar grandes volúmenes de datos ya que el costo computacional aumenta. Hierarchical Softmax, Negative Sampling y Noise Contrastive Estimation son técnicas utilizadas para optimizar este problema. Estas funciones consisten en lo siguiente:

* Hierarchical Softmax

    Esta técnica usa un árbol binario para representar las palabras, reduciendo significativamente la complejidad computacional de calcular las probabilidades de salida desde O(W)O(W) a O(log⁡2(W))O(log2​(W)). Aunque es eficiente para grandes vocabularios, su rendimiento depende mucho de cómo esté estructurado el árbol. Esto la hace muy útil para acelerar el entrenamiento, especialmente con palabras frecuentes que reciben códigos más cortos.

* Noise Contrastive Estimation (NCE)

    NCE se propone como una forma de entrenar modelos diferenciando datos reales de una distribución de ruido a través de regresión logística. Es conceptualmente más compleja y requiere ajustes precisos, pero proporciona una base teórica sólida para la estimación eficiente de los parámetros del modelo.

* Negative Sampling (NEG)

    Negative Sampling es una simplificación de Noise Contrastive Estimation que mejora la velocidad de entrenamiento al enfocarse en diferenciar la palabra objetivo de unas pocas muestras negativas en lugar de todo el vocabulario. Es menos preciso que Hierarchical Softmax para datasets pequeños pero escala maravillosamente bien para conjuntos de datos más grandes, haciendo que el entrenamiento sea viable incluso con vocabularios extremadamente grandes.


Aunque Hierarchical Softmax y NCE tienen sus méritos, Mikolov favorece el uso de Negative Sampling para la mayoría de las aplicaciones prácticas del modelo Skip-gram. NEG no solo simplifica enormemente el proceso de entrenamiento sino que también provee una eficiencia excepcional sin sacrificar la calidad de las representaciones vectoriales aprendidas. Es especialmente útil para conjuntos de datos grandes, donde un número pequeño de muestras negativas puede ser suficiente para mantener un alto rendimiento.

**5. ¿Cual diría usted que es la principal conclusión y aportación del paper de w2v? ¿Qué crítica haría usted a estos papers de w2v?**

Bueno, me parecieron increíbles. Me pareció super wao que pude leerlos y entender la mayoría sin problemas. Creo que la aportación mas importante de w2v es la capacidad de poder producir representaciones vectoriales de palabras que tengan sentido matemático y semántico. Me voló la cabeza el hecho de que las operaciones vectoriales conducían a significados semánticos. El lenguaje es algo que evoluciona en el tiempo, que no tiene muchas relaciones aparentes y que es difícil de modelar, y contar con una herramienta como word2vec, con la cual es posible fácilmente encontrar este tipo de relaciones matemáticas es increíble, me parece increíble lo que después podríamos encontrar. Mi critica es muy positiva, creo que son fáciles de leer y de entender, ademas de que citan todas las fuentes de datos que utilizaron para los entrenamientos. Lo único que me hubiera gustado es una explicación mas detallada de como se plantearon y funcionan las propuestas CBOW y skip-grams. 

## Glove

**Lea el paper de Glove y explique brevemente los siguientes puntos:**

**1. ¿Qué desventaja trata de solucionar de W2V?**

En el articulo [3], se habla acerca de los modelos de generación de espacios vectoriales semánticos. Concretamente los propuestos por Mikolov con los llamados por el autor como **Métodos de ventana poco profunda**, los cuales utilizan el contexto local para generar vectores de representación de palabras en una ventana de contexto. La desventaja que se plantea en el articulo es que mientras otros modelos como el LSA aprovechan la información estadística contenida en todo el corpus, los modelos w2v no operan con las co-ocurrencias estadísticas del corpus, sino que se limitan a utilizar una ventana a traves de todo el corpus, desaprovechando la ventaja de la cantidad de repetición de palabras en el corpus. 

**2. Describa en sus propias palabras y de manera general cual es la principal estrategia para lograrlo.**

Para poder superar las limitaciones de w2v Pennington, Jeffrey y Socher proponen GloVe por *global vectors* vectores globales. GloVe busca superar las limitaciones de Word2Vec mediante la implementación de un modelo que combina la fuerza de los métodos basados en la matriz de co-ocurrencia (LSA), que toman en cuenta la estadística del texto con la eficiencia de los modelos predictivos, (skip-grams) los cuales utilizan una ventana de contexto. GloVe utiliza una matriz global de co-ocurrencias para todo el corpus, capturando así la frecuencia con la que cada palabra aparece en el contexto de las otras. Este enfoque permite que GloVe no solo considere la información local de la ventana de contexto, sino también las estadísticas globales del corpus, solventando los problemas de cada modelo por separado, proporcionando una gran captura de las relaciones semánticas entre las palabras.

**3. Explique en sus propias palabras las principales conclusiones de los experimentos. Comente si cree que se logró el objetivo.**

Los experimentos realizados en el desarrollo de GloVe demuestran que el modelo es capaz de capturar subestructuras intrincadas del espacio semántico más eficazmente que los modelos anteriores, como Word2Vec o LSA, especialmente en tareas de analogía de palabras. Las pruebas que se hicieron en el articulo muestran que los embeddings de GloVe retienen una cantidad significativa de varianza semántica y sintáctica, lo cual era el objetivo principal. Creo firmemente que se logró el objetivo de combinar eficazmente la información global y local del corpus para producir representaciones vectoriales más informativas y precisas.

**4. ¿Encuentra alguna relación entre Glove y las clasicas TCOR y DOR? ¿Cuáles?**

Realmente si hay una relación conceptual con TCOR Y DOR en el sentido de que todos estos enfoques se basan en la observación de cómo las palabras coexisten y se distribuyen en grandes corpus de texto. Sin embargo, GloVe innova sobre estas ideas al aplicar un modelo logarítmico y una función de ponderación que ayuda a mejorar la calidad de los embeddings al ajustar la influencia de las co-ocurrencias según su frecuencia, algo que no pasa en los métodos TCOR y DOR.

**5. ¿Cual diría usted que es la principal conclusión y aportación del paper de Glove? ¿Qué crítica haría usted a este paper de Glove?**

En el articulo de GloVe la principal aportación del paper de GloVe es demostrar que es posible mejorar significativamente la calidad de los embeddings de palabras integrando la información estadística global del corpus a través de una matriz de co-ocurrencia y combinarla con redes neuronales. Una posible crítica podría ser que, a pesar de su efectividad, la construcción y almacenamiento de grandes matrices de co-ocurrencia puede ser computacionalmente muy costosa y exigente en términos de memoria, lo que podría limitar su aplicabilidad en dispositivos con recursos limitados o con corpus extremadamente grandes.

## Otros papers

**Lea los papers de Directional Word2Vec, Fast Text, el de gnome-mining, así como el paper de CNNs de Kim y conteste.**

**1. ¿Qué desventaja trata de solucionar FastText y cómo lo logra?**

Una de las desventajas de los modelos w2v es que no pueden manejar palabras que están fuera del vocabulario. FastText aborda principalmente esta desventaja en el manejo de palabras fuera del vocabulario y en la captura de información morfológica de las palabras. Para solucionar este problema FastText utiliza subpalabras o n-gramas de caracteres dentro de cada palabra :o para aprender los embeddings. Por ejemplo, para la palabra "apple", FastText considerará subcadenas como "app", "ple", "appl", "e", y otros n-gramas de caracteres, dependiendo de los parámetros seleccionados. De esta manera, incluso si la palabra completa *apple* no está presente en el vocabulario, el modelo puede construir su vector a partir de los n-gramas que sí ha visto durante el entrenamiento. Esto no solo ayuda con las palabras fuera del vocabulario sino que también permite que el modelo refleje mejor la estructura morfológica de las palabras, capturando raíces, prefijos, sufijos y otras subunidades lingüísticas.

**2. ¿Cuál sería la principal desventaja de FastText vs Word2Vec?**

Una de las principales desventajas de FastText en comparación con Word2Vec es su mayor complejidad y requerimientos computacionales. Al considerar n-gramas de caracteres además de palabras completas, FastText genera y procesa muchos más datos durante el entrenamiento. FastText puede utilizar mucha mas memoria, además de tiempo de entrenamiento que un modelo w2v. A pesar de su innovación para poder manejar palabras nuevas, estas ventajas vienen con un costo de mayor complejidad computacional y uso de recursos. 

**3. ¿Qué desventaja trata de solucionar el paper de Directional W2V y cómo lo logra? Describa brevemente las conclusiones de la sección experimental.**

El modelo Directional w2v aborda la desventaja de los modelos tradicionales w2v que no consideran la dirección en la relación entre palabras. Estos modelos propuestos por Mikolov asumen que las palabras dentro de una ventana de contexto contribuyen igualmente al significado de la palabra objetivo, sin importar su posición relativa. Esta aproximación puede no capturar completamente la naturaleza direccional de las relaciones semánticas y sintácticas en el lenguaje. Para poder resolver esta problemática, el Directional W2V introduce un mecanismo que diferencia entre las palabras antes y después de la palabra objetivo en la ventana de contexto. 

La sección experimental del paper muestra que el Directional W2V puede mejorar significativamente el rendimiento en tareas que dependen de una comprensión precisa de la estructura lingüística, como el análisis sintáctico (part of speech tagging) o la desambiguación semántica (word similarity evaluation). Las pruebas empíricas sugieren que este enfoque captura mejor las relaciones contextuales, lo que resulta en una calidad superior de los embeddings y un mejor desempeño en comparación con el modelo estándar de Word2Vec. 

**4. ¿Qué se dice acerca del análisis de complejidad del Directional w2v?**

El modelo Directional Skip-Gram (DSG) aumenta ligeramente la complejidad en comparación con el modelo Skip-Gram (SG) estándar, tanto en términos de espacio como de tiempo, debido a la necesidad de gestionar un vector adicional para la dirección. Cada palabra en el DSG requiere solo una operación más que en el SG, lo que permite que el DSG mantenga una eficiencia comparable al SG, incluso cuando se amplía el contexto. A diferencia del modelo Symmetric Skip-Gram (SSG), que incrementa significativamente la complejidad al manejar múltiples embeddings de salida por cada dirección contextual, el DSG simplifica la estructura utilizando un único vector de palabra y un vector de dirección que predice explícitamente la orientación del contexto (izquierda o derecha).

**5. En el paper de gnome-mining, ¿Qué técnicas de NLP son usadas y con que objetivo intuitivo cada una?**

En el artículo sobre genome-mining, además de algunas técnicas de machine learning y deep learning, se utilizan dos técnicas principales de procesamiento de lenguaje natural para la identificación mejorada de los clusters biosintéticos de genes (BGCs) en genomas bacterianos. Estas técnicas son las Redes Neuronales Recurrentes (RNNs) específicamente en forma de Bidirectional Long Short-Term Memory (BiLSTM) y una red neuronal de tipo skip-gram que se denomina pfam2vec. Los objetivos de las técnicas son los siguientes:

* Bidirectional Long Short-Term Memory (BiLSTM):

    La BiLSTM se emplea para capturar la información contextual de las secuencias de datos tanto en la dirección hacia adelante como hacia atrás (bidireccional). Esto es crucial en la genómica, donde la función o característica de una secuencia puede ser influenciada fuertemente por sus segmentos adyacentes. Utilizar BiLSTM permite al modelo aprender de las dependencias a largo plazo y capturar mejor la estructura y las relaciones complejas en las secuencias genómicas, lo que resulta en una identificación más precisa de BGCs.

* Pfam2vec (skip-gram model):

    Similar a w2v en NLP, pfam2vec utiliza el método skip-gram para generar representaciones vectoriales (embeddings) de las familias de proteínas (Pfam). Estas representaciones son aprendidas de la misma palabra que las palabras pero ahora para proteínas. Este enfoque facilita la identificación de patrones y relaciones funcionales entre diferentes familias de proteínas, lo que es esencial para reconocer y caracterizar BGCs en genomas bacterianos.

**6. ¿En que problemas de clasificación evaluó Kim su CNN?**

Kim evaluó su red neuronal convolucional (CNN) principalmente en la tarea de clasificación de secuencias genómicas, específicamente diferenciando entre clústeres biosintéticos de genes (BGCs) y regiones no-BGC. Esta clasificación se realizó utilizando conjuntos de datos diversificados que incluían secuencias etiquetadas como BGC y no-BGC para entrenar y validar el modelo. Los datos para la validación y el entrenamiento incluían un conjunto significativo de BGCs publicados y otros identificados en la base de datos MIBiG, junto con múltiples genomas bacterianos que habían sido manualmente anotados con regiones BGC y no-BGC.

**7. En los resultados dónde estuvo involucrado algún método de clasificación con SVM, ¿Cómo fue el resultado respecto a CNNs? ¿Qué features usaba el método basado en SVM?**

Las características típicamente empleadas en SVM para tareas similares incluyen conteos de n-gramas de aminoácidos, composición de codones, o características físico-químicas derivadas de las secuencias. En este contexto y de manera general las CNNs tienen un rendimiento superior debido a su capacidad para aprender características representativas directamente de los datos crudos, capturando relaciones complejas que los métodos basados en características predefinidas como las SVM podrían no detectar eficientemente.

**8. En sus propias palabras, ¿Qué diferencia tienen las estrategias multi-channel y single-channel?, ¿Cuál recomienda Kim?**

Las estrategias multi-channel y single-channel se refieren al uso de múltiples o un único conjunto de características o representaciones de datos, respectivamente. En una estrategia multi-channel, diferentes tipos de datos o diferentes representaciones de un mismo dato son procesados en canales separados, permitiendo que el modelo capture y combine información variada de manera efectiva. En contraste, un enfoque single-channel utiliza una única fuente de datos o representación. Aunque Kim no especifica explícitamente una recomendación entre multi-channel y single-channel, generalmente en deep learning, especialmente en tareas complejas de clasificación y predicción, los enfoques multi-channel suelen ser preferidos por su capacidad para integrar y aprender de múltiples perspectivas de los datos.

**9. ¿Cuál diría usted que es la principal conclusión y aportación del paper de Kim? ¿Qué crítica le haría usted a Kim?**

La principal conclusión y aportación del trabajo de Kim es la demostración de que las técnicas avanzadas de deep learning, como las CNNs y específicamente las arquitecturas BiLSTM y técnicas de embedding como pfam2vec, pueden ser efectivamente aplicadas para la identificación y clasificación de BGCs en genomas bacterianos. Esto marca un avance significativo en la genómica, permitiendo una detección más precisa y automática de regiones genéticas de interés. Una posible crítica al estudio podría ser la necesidad de una mayor claridad en la comparación directa de los rendimientos entre diferentes modelos de clasificación, como CNNs frente a SVM, y cómo cada modelo maneja diferentes tipos y tamaños de datos, lo que podría proporcionar insights más profundos sobre la aplicabilidad y eficiencia de las CNNs en comparación con métodos más tradicionales.

## 4 CNNs

**Estudie superficialmente el siguiente [notebook](https://github.com/fagonzalezo/dl-tau-2017-2/blob/master/Handout-CNN-sentence-classification.ipynb). En esta tarea se le pro-porcionará el CNN-rand pero en Pytorch. Investigue lo necesario para completar el CNN-static y el CNN-non-static por usted mismo. Contruya la gŕafica de comparación de los tres.**

## Referencias

[1] Mikolov, Tomas, et al. "Distributed representations of words and phrases and their compositionality." Advances in neural information processing systems 26 (2013).

[2] Mikolov, Tomas, et al. "Efficient estimation of word representations in vector space." arXiv preprint arXiv:1301.3781 (2013).

[3] Pennington, Jeffrey, Richard Socher, and Christopher D. Manning. "Glove: Global vectors for word representation." Proceedings of the 2014 conference on empirical methods in natural language processing (EMNLP). 2014.

[4] Song, Yan, et al. "Directional skip-gram: Explicitly distinguishing left and right context for word embeddings." Proceedings of the 2018 Conference of the North American Chapter of the Association for Computational Linguistics: Human Language Technologies, Volume 2 (Short Papers). 2018.

[5] Bojanowski, Piotr, et al. "Enriching word vectors with subword information." Transactions of the association for computational linguistics 5 (2017): 135-146.

[6] Hannigan, Geoffrey D., et al. "A deep learning genome-mining strategy for biosynthetic gene cluster prediction." Nucleic acids research 47.18 (2019): e110-e110.